In [1]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar, open_viewer, PartGroup, Part
from jupyter_cadquery.viewer.client import show, show_object

from types import SimpleNamespace as d

from collections.abc import Iterable
import types, copy

class WPPart(Part):
    def __init__(self, wp, cq_obj, *args, **kwargs):
        self.wp = wp
        super().__init__(cq_obj, *args, **kwargs)

    def __getattr__(self, attr):
        return getattr(self.wp, attr)
    
    def __hasattr__(self, attr):
        if attr == 'objects': return False
        return hasattr(self.cq_shape, attr)

class ProxyMethodWrapper:
    def __init__(self, obj, func, name):
        self.obj, self.func, self.name = obj, func, name

    def __call__(self, *args, **kwds):
        return self.obj._method_call(self.name, self.func, *args, **kwds)
    
class WPPartGroup(PartGroup):
    def __init__(self, wp, cq_objs, *args, **kwargs):
        self.wp = wp
        super().__init__(cq_objs, *args, **kwargs)

    def __getattr__(self, attr):
        #print("attr: ", self, attr)
        att = getattr(self.wp, attr)
        if type(att) is types.MethodType:
            return ProxyMethodWrapper(self, att, attr)
        else:
            return attr
    
    def __hasattr__(self, attr):
        return hasattr(self.wp, attr)
    
    def __repr__(self):
        return "<__main__.WPPartGroup object at " + str(id(self)) + "/" + self.name + "/" + repr(self.wp) + ">"
    
    def do(self, elems, name=None):
        #print(">>>wp/do")
        cq_wp = get_wp(self)
        if name is None: name = 'Group-Do' + ident(self)
        parts = [to_part(cq_wp, elem) for elem in elems]
        #self._append(WPPartGroup(cq_wp, [Part(self.wp.findSolid(), name = name), PartGroup(parts, name = name + ':do'), PartGroup([], 'workplane')]))
        self._append(Part(self.wp.findSolid(), name = name))
        return self

    def do_cut(self, elems, name=None):
        def cut_all(wp, part):
            if isinstance(part, PartGroup):
                return wp.cut(part.compound())
            elif isinstance(part, Part):
                return cut_all(wp, part.cq_shape)
            elif isinstance(part, Iterable):
                for p in part:
                    wp = wp.cut(p)
                return wp
            elif isinstance(part, cq.Workplane):
                return wp.cut(part.findSolid())
            else:
                return wp.cut(part)
        if name is None: name = 'Group-Cut' + ident(self)
        cq_wp = get_wp(self)
        parts = [to_part(cq_wp, elem, color = '#707070', show_faces = False) for elem in elems]
        wp = cq_wp.findSolid()
        for p in parts:
            wp = cut_all(wp, p)
        #self._append(WPPartGroup(cq_wp.newObject([wp]), [Part(wp, name = name), PartGroup(parts, name + ':do_cut'), PartGroup([], 'workplane')]))
        return self

    def _append(self, o):
        self.objects[-1].objects.append(o)

    def _method_call(self, name, func, *args, **kwargs):
        #print("call: ", self, name, args, kwargs)
        res = func(*args, **kwargs)
        self.wp = get_wp(func(*args, **kwargs))
        self._append(Part(self.wp, name + ident(self.wp)))
        return self


def new_wp(self):
    def copy_attrs(fro, to, attrs):
        for att in attrs:
            setattr(to, att, copy.copy(getattr(fro, att)))
    
    wp = self.workplane()
    wp.parent = None
    wp.ctx = cq.cq.CQContext()
    copy_attrs(self.ctx, wp.ctx, ['pendingWires', 'pendingEdges', 'firstPoint', 'tolerance', 'tags'])
    wp._tag = None
    return wp

def get_wp(wp_like):
    #print("gwp", wp_like)
    return wp_like.wp if isinstance(wp_like, PartGroup) else wp_like

def ident(o):
    return '-' + str(id(o))[-5:]

def to_part(wp, val, **kwargs):
    def make_unpacked(fn, name):
        o = fn(new_wp(wp))
        if isinstance(o, PartGroup):
            if len(o.objects) == 1: return o.objects[0]
            if name: o.name = name
            return o
        elif isinstance(o, Part):
            if name: o.name = name
            return o
        else:
            kwargs['name'] = name
            return Part(o, **kwargs)

    if isinstance(val, Iterable):
        return make_unpacked(val[0], val[1])
    return make_unpacked(val, getattr(val, 'name') if hasattr(val, 'name') else (val.__class__.__name__ + ident(val)))

def _workplane_do(self, elems, name=None):
    #print(">>>do")
    cq_wp = self
    if name is None: name = 'Group-Do' + ident(self)
    parts = [to_part(cq_wp, elem) for elem in elems]
    #print('not pg1', self)
    #return cq_wp.union(cq.Workplane([p.cq_shape for p in parts]))
    for to_union in [p.cq_shape for p in parts]:
        cq_wp = cq_wp.union(to_union)
    return cq_wp

def _workplane_do_cut(self, elems, name=None):
    if name is None: name = 'Group-Cut' + ident(self)
    cq_wp = self
    parts = [to_part(cq_wp, elem, color = '#707070', show_faces = False) for elem in elems]
    #print('not pg2', self)
    for to_cut in [p.cq_shape for p in parts]:
        cq_wp = cq_wp.cut(to_cut)
    return cq_wp
    
def _workplane_show_tree(self, name='root'):
    return WPPartGroup(self, [Part(self), PartGroup([], 'workplane')])

cq.Workplane.do = _workplane_do
cq.Workplane.do_cut = _workplane_do_cut
cq.Workplane.show_tree = _workplane_show_tree

#cq.Workplane().box(10, 10, 10).faces(">X").do_cut([
#    ((lambda wp: wp.box(2,2,2)), 'box')]).faces("<X").do_cut([((lambda wp: wp.box(2,2,2)), 'box')])

Overwriting auto display for cadquery Workplane and Shape


In [2]:
cq.Workplane().box(10, 10, 10).faces(">Z").do_cut([
    (lambda wp: wp.box(5, 5, 5), 'cut'),
    (lambda wp: wp.box(2, 2, 8), 'cut1')
]).faces(">Z").fillet(1).faces("<Z").chamfer(1).do([
    (lambda wp: wp.box(2, 2, 8), 'add2')
])

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [ ]:
def gt2_belt(l, w, oversize = 0.0):
    # veeery approximately a 2GT/GT2 belt.
    def make(wp):
        h, p, tooth_h, tooth_r = 1.38 + oversize, 2.0, 0.75 - oversize, 0.555 + 2 * oversize
        belt_h = h - tooth_h
        def make1(sketch, pos):
            x, y = pos
            x1 = x + (p - 2 * tooth_r) / 2
            x2 = x1 + 2 * tooth_r
            return (sketch
                    .segment( (x, y), (x, y + belt_h) ).segment( (x1, belt_h) ).segment( (x1, h - tooth_r) )
                    .arc( (x1, h - tooth_r), (x1 + tooth_r, h), (x2, h - tooth_r) )
                    .segment( (x2, belt_h) ).segment( (x + p, belt_h) ).segment( (x + p, y) ).close().assemble().finalize().extrude(w)
                   )
        profile = make1(wp.sketch(), (0, 0))
        res = profile
        for i in range(int(l / p) - 1):
            res = res.union(profile.translate( (p * (i + 1), 0, 0) ))
        return res
    return make

belt_d = d(w = 1.35 + 0.4, h = 6.0)

gt2_belt = gt2_belt(20, belt_d.h, oversize = 0.07)(cq.Workplane())

gt2_belt

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [3]:
not_centered = { 'centered': (False, False, False) }

def make_points(base_point, extents, offsets):
    return map(lambda p: (base_point[0] + p[0] * extents[0], base_point[1] + p[1] * extents[1]), offsets)

not_centered = { 'centered': (False, False, False) }

def make_points(base_point, extents, offsets):
    return map(lambda p: (base_point[0] + p[0] * extents[0], base_point[1] + p[1] * extents[1]), offsets)

stepper_35_d = d(w = 36, h = 36, l = 22.0, bolt_dd = 26, bolt_inset = 4.25, bolt_D = 6.2, conn = d(h = 16.5, w = 10.0, l = 7.0))
mgn12h_d = d(hole_dx = 3.5, hole_D = 2.5 + 0.5, w = 27.0, l = 45.4, bolt_dd = 20.0)
mount_d = d(mot = stepper_35_d,
            l = 36.5,
            l_tot = 36.5,
            bolt_dl = 24.5,
            bolt_dd = 15.0,
            bolt_D = 2.6,
            th_top = 2.0,
            th_bot = 3.5,
            th = 2.0,
            th_sides_extra = 4.0,
            mgn12_hole_inset = 1.0,
            fila_cog_l = 5.0,
            fila_cog_w = 20.0,
            fila_lever_top_l = 20.0,
            nozzle_bottom_w = 14.0,
            heat_sert_top_back_offs = 12.0,
           )
M2_5_screw_csk_d = d(D = 2.5 + 0.4, D_top = 5.0 + 0.2, csk_angle = 82)
M3_screw_csk_d = d(D = 3.0 * 1.2, D_top = 5.7 * 1.2, csk_angle = 82)
belt_d = d(w = 1.35 + 0.4, h = 6.0)
heat_sert_d = d(D_sm = 3.9, l = 3.0, D_lg = 5.0)

rail_top_to_carriage_bot = 16.5

#belt_attachment_d = d(th = 3.5, l = 9.8, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt_d)
#belt_attachment_front_d = d(th = 3.5, l = 6.0, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt_d)

belt_attachment_d = d(th = 3.5, l = 9.8, lower_dz = 16.5 - rail_top_to_carriage_bot, higher_dz = 8.5 - rail_top_to_carriage_bot, belt = belt_d)
belt_attachment_front_d = d(th = 3.5, l = 6.0, lower_dz = 16.5 - rail_top_to_carriage_bot, higher_dz = 8.5 - rail_top_to_carriage_bot, belt = belt_d)
def belt_mount_dz(belt_mount_d):
    return 3 * belt_mount_d.belt.w + 2 * belt_mount_d.th

In [4]:
def _workplane_centerBetween(self, direction="x"):
    if self.size() < 2: raise ValueError('Expecting two values on the stack')
    edges = getattr(self.item(0).val().Center(), direction), getattr(self.item(1).val().Center(), direction)
    newCenter = self.plane.origin
    setattr(newCenter, direction, edges[1] + (edges[0] - edges[1]) / 2.0)
    return self.newObject([newCenter])

cq.Workplane.centerBetween = _workplane_centerBetween

def belt_mount(belt_attachment_d, mount_d):
    def belt_mount_make(wp):
        b = belt_attachment_d
        m = mount_d
        w = m.mot.w + 2* m.th

        l_h, r_h = 12 - b.higher_dz, 20 - b.lower_dz
        b_w = 2 * b.th
        m_w = 2 * m.th

        #a_z = 3 * b.belt.w + 2 * b.th
        a_z = belt_mount_dz(b)
        belt_cut_z = 1.5 * b.belt.w
        belt_h = 1.1 * b.belt.h

        lower_dz, higher_dz = b.lower_dz - b.belt.h / 2 - 2 * b.th, b.higher_dz - b.belt.h / 2 - 2 * b.th

        part = (
            wp.moveTo(0, m.th_bot).sketch()
                .segment( (0, -lower_dz), (0, -m.th_bot - b.l) ).segment( (-m.th, -m.th_bot - b.l) ).segment( (-m.th, -m.th_bot) )
                .segment( (-w + m.th, -m.th_bot) )
                .segment( (-w + m.th, -m.th_bot - b.l) ).segment( (-w, -m.th_bot - b.l) ).segment( (-w, -higher_dz) )
                .segment( (-w + b_w, -higher_dz) ).segment( (-w + b_w, 0) ).segment( (-b_w, 0) ).segment( (-b_w, -lower_dz))
                .close().assemble().finalize().extrude(a_z)
                .faces("<Y").edges(">Z").chamfer(min([a_z, b.l]))
                .faces(">Y or <Y[1]").edges("|Z").fillet(2)
                .faces(">Z").tag('Z1').do_cut([
                    ((lambda wp: wp.faces(tag='Z1').edges('<X or <<X[7]').centerBetween().move(0,-b.higher_dz + belt_h / 2).circle(heat_sert_d.D_sm / 2).extrude(-a_z / 2)), 'belt_slot_r'),
                    ((lambda wp: wp.faces(tag='Z1').edges('>X or <<X[5]').centerBetween().move(0, -b.lower_dz + b.belt.h / 2).circle(heat_sert_d.D_sm / 2).extrude(-a_z / 2)), 'belt_slot_l'),
                ])
                .faces("<X").do_cut([
                    ((lambda wp: wp.moveTo(b.higher_dz -1.25 * b.belt.h / 2, a_z / 2).rect(1.25 * b.belt.h, belt_cut_z).extrude(-b_w)))#.workplane(offset = m.mot.w / 2).moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, 2 * belt_cut_z).cutBlind(-2 * b.th)), 'belt_slot_r')
                ])
                .faces(">X").do_cut([
                    ((lambda wp: wp.moveTo(-b.lower_dz + 1.1 * b.belt.h / 2, a_z / 2).rect(1.1 * b.belt.h, belt_cut_z).extrude(-b_w-5)))#.workplane(offset = m.mot.w / 2).moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, 2 * belt_cut_z).cutBlind(-2 * b.th)), 'belt_slot_r')
                ])
        )
        return part.mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0)).translate( (w, 0, 0) ).mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0)).translate( (w, 0, 0) )
    return belt_mount_make

def belt_mount_front(belt_attachment_d, mount_d):
    b = belt_attachment_d
    m = mount_d
    w = m.mot.w + 2* m.th
    def belt_mount_make(wp):
        return belt_mount(belt_attachment_d, mount_d)(wp.workplane(offset=-3)).mirror(mirrorPlane="XY", basePointVector=(0, 0, 0)).mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0)).translate( (w, 0, mount_d.l) )
    return belt_mount_make

#belt_mount(belt_attachment_d, mount_d)(cq.Workplane())

In [29]:
def make_mount(mount = mount_d, rail_mount_fasteners = M3_screw_csk_d, rail_top_to_carriage_bot = rail_top_to_carriage_bot, rail = mgn12h_d, belt = belt_d):
    m = mount
    w, h = m.mot.w + 2* m.th, m.mot.h + m.th_top + m.th_bot
    rail_mount_pts = make_points( (w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])        

    return  (
        cq.Workplane().box(w, h, m.l, **not_centered)
            .faces(">Z").workplane().moveTo(w/2, h/2 + ( m.th_top - m.th_bot)/2).rect(m.mot.w, m.mot.h).cutThruAll().edges("|Z").chamfer(2)
            .faces("<X").workplane().moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    .moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2 - m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    # motor wire cut-out right:
                                    # .moveTo(-m.th_top - (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
            .faces(">X").workplane().moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    .moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2 + m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    # motor wire cut-out left:
                                    .moveTo( m.th_top + (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
            .faces(">Y[1]").workplane().pushPoints(make_points( (w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).circle(rail_mount_fasteners.D / 2 + 0.5).cutBlind(-m.th_top)
            .faces(">Y[4]").tag('top-2').workplane().pushPoints( make_points( (- w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).cskHole(rail_mount_fasteners.D, rail_mount_fasteners.D_top, rail_mount_fasteners.csk_angle, m.th_bot)
                .faces(">X").workplane()
                                    # fila advance cog cut-out:
                                    .moveTo( m.th_top -(m.mot.h) / 2, -m.fila_cog_l / 2).rect(m.fila_cog_w, m.fila_cog_l).cutBlind(-m.mot.conn.l)
            .faces(">Y").tag('top').end()
            .faces("<Y").do_cut([
                # Nozzle bottom cutout.
                (lambda wp: wp.moveTo(-w / 2, -1).slot2D(m.nozzle_bottom_w, 10).extrude(-m.th_top)),
            ])
            .faces(">Z").do([
                (belt_mount(belt_attachment_front_d, mount_d), 'belt_mount_front'),
                (belt_mount_front(belt_attachment_d, mount_d), 'belt_mount_rear')
            ], '>Z')
            .faces(tag='top').do_cut([
                # Cutout for filament lever.
                (lambda wp: wp.moveTo(w -  m.fila_lever_top_l / 2 - 2 * belt_attachment_d.th , 8).rect(m.fila_lever_top_l, 16).extrude(-m.th_bot)),
                # Top back heatserts.
                #(lambda wp: wp.moveTo(w / 2 - m.heat_sert_top_back_offs / 2 - m.th, -m.l - 2 * belt_attachment_d.belt.w).lineTo(w / 2  + m.heat_sert_top_back_offs / 2 - m.th, -m.l - 2 * belt_attachment_d.belt.w).vertices().circle(heat_sert_d.D_sm / 2).extrude(-m.th_bot)),
            ])
            # "Thicken" sides.
            .faces("<X").wires().toPending().extrude(-m.th_sides_extra)
            .faces(">X").wires().toPending().extrude(m.th_sides_extra)
            # Cut rear belt slot.
            .faces(tag='top-2').workplane().do_cut([
                (lambda wp: wp.center(-w + 2*belt_attachment_d.th + 1.5 * belt_attachment_d.belt.w, -m.l - belt_attachment_d.belt.h).rect(-3*belt_attachment_d.belt.w, belt_attachment_d.belt.h + 2).extrude(-m.th_bot)),
            ])
            .faces(tag='top-2').workplane().moveTo(-(w / 2 - m.heat_sert_top_back_offs / 2 - m.th), -m.l - 2 * belt_attachment_d.belt.w).lineTo(-(w / 2  + m.heat_sert_top_back_offs / 2 - m.th), -m.l - 2 * belt_attachment_d.belt.w).vertices().cskHole(rail_mount_fasteners.D, rail_mount_fasteners.D_top, rail_mount_fasteners.csk_angle, m.th_bot)

    )

tooldoard_d = d(hole_offs_x = 45.0, offs_z = 25.0, th = 3.0)

def toolboard_mount(tooldoard = tooldoard_d, mount = mount_d, rail_mount_fasteners = M3_screw_csk_d, rail_top_to_carriage_bot = rail_top_to_carriage_bot, rail = mgn12h_d, belt = belt_d):
    m = mount
    tb = tooldoard_d
    
    w, h = m.mot.w + 2* m.th, m.mot.h + m.th_top + m.th_bot
    ww = w - (w / 2 - m.heat_sert_top_back_offs / 2 - m.th) - 2 * belt_attachment_d.th
    hh = belt_mount_dz(belt_attachment_d)
    ll = tb.hole_offs_x + 10

    return (
        cq.Workplane("XZ").workplane(offset=-h).center((w / 2 - m.heat_sert_top_back_offs / 2 - m.th), -m.l - 2 * belt_attachment_d.belt.w + m.l/2)
            .box(ww , hh, heat_sert_d.l, **not_centered)
            # holes 
            .moveTo(rail_mount_fasteners.D + 1, hh/2).lineTo(rail_mount_fasteners.D + 1 + w/2 - m.heat_sert_top_back_offs / 2 - m.th, hh/2).vertices().circle(heat_sert_d.D_sm / 2).cutBlind(heat_sert_d.l)
            .faces(">Y").center(ww, 0).workplane().rect(8, tb.th, centered=(False, False)).workplane(offset=tb.offs_z).rect(ww, tb.th, centered=(False, False)).loft()
            .faces(">Y[-2]").edges(">X[2]").fillet(3)
            .faces(">Y").edges("<X").workplane().rect(ll, w / 2, centered=(False, False)).extrude(tb.th).faces(">Y[-2]").edges(">X[2]").fillet(6).edges("|Y").fillet(2)
            .faces(">Y").edges("<X").workplane().moveTo(tb.hole_offs_x, w/4).circle(heat_sert_d.D_sm/2).cutBlind(-tb.th)
        )
mount = make_mount()
toolboard = toolboard_mount()

PartGroup([Part(mount), Part(toolboard)])

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [16]:
from cadquery import exporters

exporters.export(mount, 'biqu_h2o_mg12h_mount_v3_full.stl')

In [117]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar

from types import SimpleNamespace as d
from jupyter_cadquery.viewer.client import show, show_object

not_centered = { 'centered': (False, False, False) }

# tringular_section_d = d(w = 13.3, l = 13.3, th_bot = 2.75, h_rect = 4, h = 11.45, hole_D = 3.2, hole_edge_dh = 6.31, hole_center_dh = 8.0, w_top = 7.8, th_side_r = 2.1, th_side_l = 1.5)
# lever_d = d(w = 4.6, l = 28.0, h = 8.5, hole_D = 4.7, hole_dl = 1.5)

# Adjusted to my printer/filament's dimensions.
tringular_section_d = d(w = 13.3, l = 13.3, th_bot = 2.75, h_rect = 4, h = 11.45, hole_D = 3.5, hole_edge_dh = 6.31, hole_center_dh = 8.0, w_top = 7.8, th_side_r = 2.1, th_side_l = 1.5)
lever_d = d(w = 5.1, l = 34.0, h = 8.5, hole_D = 4.95, hole_dl = 1.5, spring_retainer = d(i_D = 4.0, o_D = 7.0, d = 1.0))

t = tringular_section_d
l = lever_d

lever_triangular = (
    cq.Workplane("YZ")
                    .sketch()
                        .segment( (0, 0), (t.w, 0) ).segment( (t.w, t.h_rect) ).segment( (t.w / 2 + t.w_top / 2, t.hole_center_dh) ).arc((t.w / 2, t.hole_center_dh), t.w_top / 2, 0., 360).segment( (t.w / 2 - t.w_top / 2, t.hole_center_dh) ).segment( (0, t.h_rect) ).close().hull()
                        .push( [(t.w / 2, t.hole_center_dh)] ).circle(t.hole_D / 2, mode = 's').finalize()
                    .extrude(t.l)
                    .faces("<Y").workplane().moveTo((t.th_side_r - t.th_side_l) / 2 + t.w / 2, t.h/2 + t.th_bot).rect(t.w - t.th_side_r - t.th_side_l, t.h).cutThruAll()
)
face = lever_triangular.faces("<Y").tag('joint')
lever_triangular = lever_triangular.workplaneFromTagged('joint').moveTo(l.w/2, l.h/2).rect(l.w, l.h).workplane(offset=l.l).moveTo(l.w/2, l.h/2).rect(l.w, l.h/2).loft()
lever_triangular = lever_triangular.workplaneFromTagged('joint').moveTo(t.th_side_r/2, l.h/2).rect(t.th_side_r, l.h).extrude(-t.hole_center_dh/2).faces("<Y[1]").edges("<X").fillet(1.5)
lever_triangular = lever_triangular.workplaneFromTagged('joint').transformed(rotate=(0, 90, 0), offset=(0, l.h/2, l.hole_D/2 + l.hole_dl)).circle(l.hole_D/2).cutBlind(l.w)
lever_triangular = lever_triangular.faces("<Y").fillet(2)
lever_triangular = lever_triangular.faces("<Z").do_cut([ (lambda wp: wp.moveTo(l.spring_retainer.o_D / 2 + 3, -l.l / 2 + l.spring_retainer.o_D / 2 + 1.5).circle(l.spring_retainer.i_D / 2).circle(l.spring_retainer.o_D / 2).extrude(-l.spring_retainer.d)) ])
lever_triangular

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [53]:
from cadquery import exporters

exporters.export(lever_triangular, 'biqu_h2o_lever_slim.stl')

In [346]:
# Schmidt X/Y Proto PCB mount.

#pcb_d = d(l = 62.0, w = 23.5, rail_hole2hole = 50.0, pcb_hole_D = 5.0, screw_hole_D = 3.0, l_from_hole_ctr = 45.5, w_from_hole_ctr = 22.0, th = 2.0, l_supp = 20, d = 1.95)

def pcb_mount1(pcb = pcb_d):    
    mount = (
        cq.Workplane().sketch()
            .arc((0, 0), pcb.pcb_hole_D, 90, 180).arc((0, 0), pcb.pcb_hole_D / 2, 0, 360).arc((pcb.rail_hole2hole, 0), pcb.screw_hole_D, -90, 180).arc((pcb.rail_hole2hole, 0), pcb.screw_hole_D / 2, 0, 360)
            .segment((0, -pcb.pcb_hole_D), (pcb.rail_hole2hole, -pcb.screw_hole_D)).segment((pcb.rail_hole2hole, pcb.screw_hole_D), (0, pcb.pcb_hole_D))
            .assemble().finalize().extrude(2)
            .edges("%CIRCLE and >>X[4]").tag('circle2').end()
            .do([(
                lambda wp: wp.workplane(offset=-1).moveTo(pcb.l_from_hole_ctr- pcb.l_supp / 2, pcb.w_from_hole_ctr - pcb.w).rect(pcb.l_supp, pcb.w + 2 * pcb.th, centered = (False, False)).extrude(4 + pcb.d)
                    .faces(">X").workplane().moveTo(0, 2).rect(pcb.w, pcb.d, **not_centered).cutBlind(-pcb.l_supp).moveTo(0, 2 + pcb.d).rect(pcb.w - 2, 2, **not_centered).cutBlind(-pcb.l_supp)
            )])
            .workplaneFromTagged('circle2').moveTo(pcb.rail_hole2hole, 0).circle(pcb.screw_hole_D / 2).cutBlind(pcb.th)
            .workplaneFromTagged('circle2').circle(pcb.pcb_hole_D / 2).circle(pcb.screw_hole_D / 2).extrude(pcb.th + 1.5 * pcb.d).mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0))
    )
    return mount

pcb_d = d(l = 62.0, w = 24.5, rail_hole2hole = 50.0, pcb_hole_D = 5.0 - 0.05, screw_hole_D = 3.0 + 0.1, main_w = 7.0, l_from_hole_ctr = 45.5, w_from_hole_ctr = 22.0, th = 2.0, l_supp = 20, d = 2.3, conn_h = 4.0,
          offset = d(x = 5.5, y = 2 * 5.0 - 2.0, y_bot = 11.9), plug_45_pcb_hole_offs = d(x = 5.0, y = 3.0))

def pcb_mount(pcb = pcb_d):
    hole_D_2 = pcb.pcb_hole_D / 2
    mount = (
        cq.Workplane().sketch()
            .arc((0, 0), pcb.main_w / 2, 90, 180).arc((0, 0), pcb.screw_hole_D / 2, 0, 360).arc((pcb.rail_hole2hole, 0), pcb.main_w / 2, -90, 180).arc((pcb.rail_hole2hole, 0), pcb.screw_hole_D / 2, 0, 360)
            .segment((0, -pcb.main_w / 2), (pcb.rail_hole2hole, -pcb.main_w / 2)).segment((pcb.rail_hole2hole, pcb.main_w / 2), (0, pcb.main_w / 2))
            .assemble().finalize().extrude(pcb.conn_h)
            .edges("%CIRCLE and >>X[4]").tag('circle2').end()
            .faces(">Z").edges('not %CIRCLE').chamfer(1.8).workplane(centerOption='CenterOfBoundBox')
            .do([(
                lambda wp: wp.workplane(offset=-pcb.conn_h / 2).center(-pcb.rail_hole2hole / 2, 0)
                            # Narrow rapezoid extension with peg for PCB.
                            .moveTo(0, - hole_D_2).sketch().segment( (0, 0,), (pcb.offset.x - hole_D_2, -pcb.offset.y + hole_D_2) ).segment( (pcb.offset.x + hole_D_2, -pcb.offset.y + hole_D_2) ).segment( (2 * pcb.offset.x, 0) ).close().assemble().finalize().extrude(pcb.conn_h - 1)
                            .moveTo(pcb.offset.x, -pcb.offset.y).circle(pcb.pcb_hole_D / 2).circle(pcb.screw_hole_D / 2).extrude(pcb.conn_h + 1.5 * pcb.d)
                            .moveTo(pcb.offset.x, -pcb.offset.y).circle(pcb.screw_hole_D / 2).cutThruAll()
                            # Small cutout for endstop soldered header.
                            #.do_cut([( lambda wp: (lambda x, y, offs: wp.workplane(offset = -1).moveTo(x, y).rect(4, 4).extrude(-4).rotate( (x, y, 0), (x, y, 1), 45)) (x = pcb.offset.x + hole_D_2 + pcb.plug_45_pcb_hole_offs.x, y = -pcb.offset.y + hole_D_2 + pcb.plug_45_pcb_hole_offs.y, offs = pcb.plug_45_pcb_hole_offs) )])
                            # Wide trapezoid with edge.
                            .moveTo(pcb.rail_hole2hole - pcb.screw_hole_D, pcb.pcb_hole_D / 2).sketch().segment( (0, -pcb.main_w/2), (-10, pcb.offset.y_bot + pcb.th / 2) ).segment( (-30, pcb.offset.y_bot + pcb.th / 2) ).segment( (-40, -pcb.main_w/2) ).close().assemble().finalize().extrude(pcb.conn_h)
                            .moveTo(pcb.rail_hole2hole - pcb.screw_hole_D, pcb.pcb_hole_D / 2 +  pcb.offset.y_bot).sketch().segment( (-10, 0), (-10, pcb.th / 2) ).segment( (-30, pcb.th / 2) ).segment( (-30, 0) ).close().assemble().finalize().extrude(pcb.conn_h + pcb.th)
                            .faces(">Y").edges("<Z").chamfer(3)
            )])
            #.do([(
            #    lambda wp: wp.workplane(offset=-1).moveTo(pcb.l_from_hole_ctr- pcb.l_supp / 2, pcb.w_from_hole_ctr - pcb.w).rect(pcb.l_supp, pcb.w + 2 * pcb.th, centered = (False, False)).extrude(4 + pcb.d)
            #        .faces(">X").workplane().moveTo(0, 2).rect(pcb.w, pcb.d, **not_centered).cutBlind(-pcb.l_supp).moveTo(0, 2 + pcb.d).rect(pcb.w - 2, 2, **not_centered).cutBlind(-pcb.l_supp)
            #)])
            .workplaneFromTagged('circle2').moveTo(pcb.rail_hole2hole, 0).circle(pcb.screw_hole_D / 2).cutBlind(pcb.conn_h)
            
    )
    return mount

pcb_mount = pcb_mount()

pcb_mount

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [347]:
from cadquery import exporters

exporters.export(pcb_mount, 'sapphire_plus_pcb_xy_mount.stl')